# Bài tập 1: Chuyển đổi biểu thức Trung tố sang Hậu tố

**Mục tiêu:** Minh họa thuật toán chuyển biểu thức từ dạng trung tố (Infix) sang dạng hậu tố (Postfix) sử dụng thuật toán Shunting-yard.

**Các toán tử được hỗ trợ:**
- Toán tử số học: `+`, `-`, `*`, `/`, `%` (chia lấy dư), `^` (lũy thừa)
- Hàm toán học: `sin`, `cos`, `tan`, `sqrt`
- Toán tử đơn ngôi: `!` (giai thừa)

### 1.1. Các dạng biểu thức toán học

| Dạng | Tên gọi | Ví dụ | Đặc điểm |
|------|---------|-------|----------|
| **Trung tố** | Infix | `A + B` | Toán tử nằm **giữa** hai toán hạng |
| **Hậu tố** | Postfix (RPN) | `A B +` | Toán tử nằm **sau** các toán hạng |
| **Tiền tố** | Prefix | `+ A B` | Toán tử nằm **trước** các toán hạng |

### 1.2. Tại sao cần chuyển sang Hậu tố?

**Biểu thức trung tố** (dạng con người thường dùng) có nhược điểm:
- Cần quy tắc độ ưu tiên toán tử (nhân chia trước, cộng trừ sau)
- Cần dấu ngoặc để thay đổi thứ tự tính toán
- Máy tính khó xử lý trực tiếp

**Biểu thức hậu tố** có ưu điểm:
- Không cần dấu ngoặc
- Không cần quy tắc độ ưu tiên
- Dễ dàng tính toán bằng Stack (chỉ cần duyệt từ trái sang phải)

### 1.3. Ví dụ chuyển đổi

| Trung tố (Infix) | Hậu tố (Postfix) |
|------------------|------------------|
| `3 + 4` | `3 4 +` |
| `3 + 4 * 2` | `3 4 2 * +` |
| `(3 + 4) * 2` | `3 4 + 2 *` |

---

## 2. Thuật toán Shunting-yard

Thuật toán **Shunting-yard** (do Edsger Dijkstra phát minh) sử dụng một **Stack** để chuyển đổi biểu thức trung tố sang hậu tố.

### 2.1. Cấu trúc dữ liệu

| Thành phần | Mô tả |
|------------|-------|
| **Input** | Chuỗi biểu thức trung tố |
| **Output Queue** | Danh sách chứa kết quả hậu tố |
| **Operator Stack** | Stack tạm chứa các toán tử chờ xử lý |
| **Precedence Table** | Bảng độ ưu tiên của các toán tử |

### 2.2. Bảng độ ưu tiên toán tử

| Độ ưu tiên | Toán tử | Ghi chú |
|------------|---------|---------|
| 5 | `sin`, `cos`, `tan`, `sqrt` | Hàm số |
| 4 | `!` | Giai thừa |
| 3 | `^` | Lũy thừa (kết hợp phải) |
| 2 | `*`, `/`, `%` | Nhân, chia, chia lấy dư |
| 1 | `+`, `-` | Cộng, trừ |
| 0 | `(` | Ngoặc mở (đặc biệt) |

### 2.3. Các bước thực hiện

**Bước 1: Tokenization (Tách token)**
- Tách chuỗi đầu vào thành các token riêng biệt (số, biến, toán tử, ngoặc, hàm)

**Bước 2: Duyệt từng token**

| Token là | Hành động |
|----------|-----------|
| **Số hoặc biến** | Đưa thẳng vào Output |
| **Hàm số** (`sin`, `cos`...) | Đẩy vào Stack |
| **Ngoặc mở** `(` | Đẩy vào Stack |
| **Ngoặc đóng** `)` | Pop Stack → Output cho đến khi gặp `(`, sau đó kiểm tra nếu đỉnh Stack là hàm thì pop luôn |
| **Toán tử** | Pop các toán tử có độ ưu tiên ≥ ra Output, rồi đẩy toán tử hiện tại vào Stack |

**Bước 3: Kết thúc**
- Pop tất cả toán tử còn lại trong Stack ra Output

### 2.4. Ví dụ minh họa chi tiết

**Input:** `3 + sin(x) * 2`

| Bước | Token | Stack | Output | Giải thích |
|------|-------|-------|--------|------------|
| 1 | `3` | `[]` | `3` | Số → ra Output |
| 2 | `+` | `[+]` | `3` | Stack rỗng → vào Stack |
| 3 | `sin` | `[+, sin]` | `3` | Hàm → vào Stack |
| 4 | `(` | `[+, sin, (]` | `3` | Ngoặc mở → vào Stack |
| 5 | `x` | `[+, sin, (]` | `3 x` | Biến → ra Output |
| 6 | `)` | `[+]` | `3 x sin` | Pop đến `(`, pop `sin` |
| 7 | `*` | `[+, *]` | `3 x sin` | `*` > `+` → vào Stack |
| 8 | `2` | `[+, *]` | `3 x sin 2` | Số → ra Output |
| 9 | *Kết thúc* | `[]` | `3 x sin 2 * +` | Pop hết Stack |

**Kết quả:** `3 x sin 2 * +`

---

## 3. Cài đặt chương trình

### 3.1. Hàm chuyển đổi Infix → Postfix

In [ ]:
import re

def infix_to_postfix(expression, verbose=False):
    """
    Chuyển biểu thức từ dạng trung tố (infix) sang hậu tố (postfix).
    
    Hỗ trợ các toán tử:
    - Số học: +, -, *, /, %, ^
    - Hàm: sin, cos, tan, sqrt
    - Giai thừa: ! (postfix unary operator)
    
    Args:
        expression: Biểu thức trung tố dạng chuỗi
        verbose: Nếu True, in ra từng bước xử lý
    
    Returns:
        Biểu thức hậu tố dạng chuỗi
    """
    
    # 1. Định nghĩa độ ưu tiên
    precedence = {
        'sin': 5, 'cos': 5, 'tan': 5, 'sqrt': 5,  # Hàm số
        '!': 4,   # Giai thừa (unary postfix)
        '^': 3,   # Lũy thừa
        '*': 2, '/': 2, '%': 2,  # Nhân, chia, chia lấy dư
        '+': 1, '-': 1,  # Cộng, trừ
        '(': 0   # Ngoặc mở (thấp nhất)
    }
    
    # Danh sách các hàm số
    functions = {'sin', 'cos', 'tan', 'sqrt'}
    
    # 2. Tokenization - Tách chuỗi thành các token
    # Pattern: số (gồm thập phân) | tên hàm/biến | các ký tự đặc biệt
    tokens = re.findall(r'(\d+\.?\d*|[a-zA-Z]+|[+\-*/^%!():])', expression.replace(" ", ""))
    
    output_queue = []      # Hàng đợi kết quả
    operator_stack = []    # Stack chứa toán tử
    
    # Hàm kiểm tra số
    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    if verbose:
        print(f"\n{'='*70}")
        print(f"Biểu thức gốc: {expression}")
        print(f"{'='*70}")
        print(f"{'Token':<10} | {'Stack':<25} | {'Output':<25} | Giải thích")
        print("-" * 70)

    # 3. Duyệt qua từng token
    for token in tokens:
        explanation = ""
        
        # TRƯỜNG HỢP 1: Là số
        if is_number(token):
            output_queue.append(token)
            explanation = "Số -> đưa thẳng vào Output"
            
        # TRƯỜNG HỢP 2: Là biến (x, y, z...) nhưng không phải hàm
        elif token.isalpha() and token not in precedence and token not in functions:
            output_queue.append(token)
            explanation = "Biến -> đưa thẳng vào Output"
            
        # TRƯỜNG HỢP 3: Là hàm số (sin, cos, tan, sqrt)
        elif token in functions:
            operator_stack.append(token)
            explanation = "Hàm -> đẩy vào Stack"
            
        # TRƯỜNG HỢP 4: Ngoặc mở '('
        elif token == '(':
            operator_stack.append(token)
            explanation = "Ngoặc mở -> đẩy vào Stack"
            
        # TRƯỜNG HỢP 5: Ngoặc đóng ')'
        elif token == ')':
            # Pop tất cả toán tử cho đến khi gặp '('
            while operator_stack and operator_stack[-1] != '(':
                output_queue.append(operator_stack.pop())
            
            if not operator_stack:
                raise ValueError("Lỗi: Thiếu ngoặc mở!")
            
            operator_stack.pop()  # Bỏ '(' khỏi stack
            
            # Nếu đỉnh stack là hàm số -> pop hàm đó ra output
            if operator_stack and operator_stack[-1] in functions:
                output_queue.append(operator_stack.pop())
            
            explanation = "Ngoặc đóng -> pop đến '(', kiểm tra hàm"
        
        # TRƯỜNG HỢP 6: Toán tử giai thừa '!' (unary postfix)
        elif token == '!':
            # Giai thừa áp dụng ngay lập tức cho toán hạng trước nó
            output_queue.append(token)
            explanation = "Giai thừa -> đưa thẳng vào Output"
            
        # TRƯỜNG HỢP 7: Các toán tử khác (+, -, *, /, %, ^)
        elif token in precedence:
            # Xử lý kết hợp phải cho lũy thừa
            if token == '^':
                # Right associative: chỉ pop nếu độ ưu tiên đỉnh stack > token hiện tại
                while (operator_stack and 
                       operator_stack[-1] in precedence and 
                       precedence[operator_stack[-1]] > precedence[token]):
                    output_queue.append(operator_stack.pop())
            else:
                # Left associative: pop nếu độ ưu tiên đỉnh stack >= token hiện tại
                while (operator_stack and 
                       operator_stack[-1] in precedence and 
                       precedence[operator_stack[-1]] >= precedence[token]):
                    output_queue.append(operator_stack.pop())
            
            operator_stack.append(token)
            explanation = f"Toán tử '{token}' (ưu tiên {precedence[token]}) -> so sánh và đẩy vào Stack"
        
        if verbose:
            stack_str = str(operator_stack) if operator_stack else "[]"
            output_str = " ".join(output_queue) if output_queue else ""
            print(f"{token:<10} | {stack_str:<25} | {output_str:<25} | {explanation}")

    # 4. Pop tất cả toán tử còn lại trong Stack
    while operator_stack:
        op = operator_stack.pop()
        if op == '(':
            raise ValueError("Lỗi: Thiếu ngoặc đóng!")
        output_queue.append(op)
        
        if verbose:
            stack_str = str(operator_stack) if operator_stack else "[]"
            output_str = " ".join(output_queue)
            print(f"{'(end)':<10} | {stack_str:<25} | {output_str:<25} | Pop '{op}' còn lại")

    result = " ".join(output_queue)
    
    if verbose:
        print("-" * 70)
        print(f"KẾT QUẢ: {result}")
        print(f"{'='*70}\n")
    
    return result


# ========================
# MINH HỌA CHI TIẾT
# ========================

print("╔" + "═"*68 + "╗")
print("║" + " MINH HỌA THUẬT TOÁN CHUYỂN TRUNG TỐ SANG HẬU TỐ (Shunting-yard)".center(68) + "║")
print("╚" + "═"*68 + "╝")

# Ví dụ 1: Biểu thức đơn giản
infix_to_postfix("3 + 4 * 2", verbose=True)

# Ví dụ 2: Có ngoặc
infix_to_postfix("( 1 + 2 ) * 3", verbose=True)

# Ví dụ 3: Có hàm số
infix_to_postfix("sin(30) + cos(60)", verbose=True)

### 3.2. Thêm ví dụ với các toán tử khác

Các ví dụ sau minh họa thêm với giai thừa (`!`), căn bậc hai (`sqrt`), lũy thừa (`^`) và chia lấy dư (`%`):

In [ ]:
# ========================
# THÊM VÍ DỤ VỚI CÁC TOÁN TỬ KHÁC
# ========================

# Ví dụ 4: Giai thừa
infix_to_postfix("5! + 3", verbose=True)

# Ví dụ 5: Căn bậc hai và lũy thừa
infix_to_postfix("sqrt(16) + 2 ^ 3", verbose=True)

# Ví dụ 6: Biểu thức phức tạp với nhiều toán tử
infix_to_postfix("3 + 4 * 2 / ( 1 - 5 ) ^ 2", verbose=True)

# Ví dụ 7: Modulo (chia lấy dư)
infix_to_postfix("10 % 3 + 2 * 5", verbose=True)

### 3.3. Bảng tổng hợp các ví dụ

Bảng dưới đây tổng hợp kết quả chuyển đổi cho nhiều loại biểu thức khác nhau:

In [ ]:
# ========================
# BẢNG TỔNG HỢP CÁC VÍ DỤ
# ========================

print("\n" + "╔" + "═"*78 + "╗")
print("║" + " BẢNG TỔNG HỢP CHUYỂN ĐỔI TRUNG TỐ → HẬU TỐ ".center(78) + "║")
print("╚" + "═"*78 + "╝")

test_expressions = [
    # Biểu thức cơ bản
    "3 + 4",
    "3 - 4",
    "3 * 4",
    "3 / 4",
    
    # Biểu thức có độ ưu tiên
    "3 + 4 * 2",
    "3 * 4 + 2",
    "2 + 3 * 4 - 5",
    
    # Biểu thức có ngoặc
    "( 3 + 4 ) * 2",
    "3 * ( 4 + 2 )",
    "( 1 + 2 ) * ( 3 + 4 )",
    
    # Lũy thừa (right associative)
    "2 ^ 3",
    "2 ^ 3 ^ 2",
    
    # Modulo
    "10 % 3",
    "10 % 3 + 5",
    
    # Giai thừa
    "5!",
    "5! + 3",
    "3 + 5!",
    
    # Hàm số
    "sin(30)",
    "cos(60)",
    "tan(45)",
    "sqrt(16)",
    
    # Biểu thức phức tạp với hàm
    "sin(30) + cos(60)",
    "sqrt(16) * 2 + 1",
    "sin(x) + cos(y) * 2",
    
    # Biểu thức tổng hợp
    "3 + sin(x) * 2",
    "sqrt(a + b) * c",
    "2 ^ 3 + 4 * 5",
    "( a + b ) * ( c - d ) / e",
]

print(f"\n{'STT':<4} | {'TRUNG TỐ (INFIX)':<35} | {'HẬU TỐ (POSTFIX)':<35}")
print("-" * 80)

for i, exp in enumerate(test_expressions, 1):
    try:
        postfix = infix_to_postfix(exp)
        print(f"{i:<4} | {exp:<35} | {postfix:<35}")
    except Exception as e:
        print(f"{i:<4} | {exp:<35} | LỖI: {str(e)}")

---

## 4. Tính giá trị biểu thức hậu tố

Sau khi chuyển sang dạng hậu tố, ta có thể dễ dàng tính giá trị biểu thức bằng Stack.

### 4.1. Thuật toán

1. Duyệt từng token từ trái sang phải
2. Nếu là **số**: đẩy vào Stack
3. Nếu là **toán tử hai ngôi** (`+`, `-`, `*`, `/`, `%`, `^`):
   - Pop hai toán hạng từ Stack
   - Thực hiện phép tính
   - Đẩy kết quả vào Stack
4. Nếu là **toán tử một ngôi** (`!`, `sin`, `cos`, `tan`, `sqrt`):
   - Pop một toán hạng từ Stack
   - Thực hiện phép tính
   - Đẩy kết quả vào Stack
5. Kết quả cuối cùng là phần tử duy nhất còn lại trong Stack

### 4.2. Ví dụ

| Biểu thức hậu tố | Quá trình tính | Kết quả |
|------------------|----------------|---------|
| `3 4 +` | 3 + 4 | 7 |
| `3 4 2 * +` | 3 + (4 * 2) = 3 + 8 | 11 |
| `5 !` | 5! = 120 | 120 |

### 4.3. Cài đặt hàm tính giá trị

In [ ]:
import math

def evaluate_postfix(postfix_expr, verbose=False):
    """
    Tính giá trị biểu thức hậu tố.
    
    Args:
        postfix_expr: Biểu thức hậu tố (chuỗi các token cách nhau bởi dấu cách)
        verbose: Nếu True, in ra từng bước tính toán
    
    Returns:
        Giá trị số của biểu thức
    """
    tokens = postfix_expr.split()
    stack = []
    
    if verbose:
        print(f"\n{'='*60}")
        print(f"Tính giá trị biểu thức hậu tố: {postfix_expr}")
        print(f"{'='*60}")
        print(f"{'Token':<10} | {'Stack':<30} | Giải thích")
        print("-" * 60)
    
    for token in tokens:
        explanation = ""
        
        # Nếu là số
        try:
            num = float(token)
            stack.append(num)
            explanation = f"Đẩy {num} vào Stack"
        except ValueError:
            # Là toán tử
            if token == '+':
                b, a = stack.pop(), stack.pop()
                result = a + b
                stack.append(result)
                explanation = f"{a} + {b} = {result}"
                
            elif token == '-':
                b, a = stack.pop(), stack.pop()
                result = a - b
                stack.append(result)
                explanation = f"{a} - {b} = {result}"
                
            elif token == '*':
                b, a = stack.pop(), stack.pop()
                result = a * b
                stack.append(result)
                explanation = f"{a} * {b} = {result}"
                
            elif token == '/':
                b, a = stack.pop(), stack.pop()
                result = a / b
                stack.append(result)
                explanation = f"{a} / {b} = {result}"
                
            elif token == '%':
                b, a = stack.pop(), stack.pop()
                result = a % b
                stack.append(result)
                explanation = f"{a} % {b} = {result}"
                
            elif token == '^':
                b, a = stack.pop(), stack.pop()
                result = a ** b
                stack.append(result)
                explanation = f"{a} ^ {b} = {result}"
                
            elif token == '!':
                a = stack.pop()
                result = math.factorial(int(a))
                stack.append(result)
                explanation = f"{int(a)}! = {result}"
                
            elif token == 'sin':
                a = stack.pop()
                result = math.sin(math.radians(a))
                stack.append(result)
                explanation = f"sin({a}°) = {result:.6f}"
                
            elif token == 'cos':
                a = stack.pop()
                result = math.cos(math.radians(a))
                stack.append(result)
                explanation = f"cos({a}°) = {result:.6f}"
                
            elif token == 'tan':
                a = stack.pop()
                result = math.tan(math.radians(a))
                stack.append(result)
                explanation = f"tan({a}°) = {result:.6f}"
                
            elif token == 'sqrt':
                a = stack.pop()
                result = math.sqrt(a)
                stack.append(result)
                explanation = f"sqrt({a}) = {result}"
        
        if verbose:
            stack_str = str([round(x, 6) if isinstance(x, float) else x for x in stack])
            print(f"{token:<10} | {stack_str:<30} | {explanation}")
    
    final_result = stack[0]
    
    if verbose:
        print("-" * 60)
        print(f"KẾT QUẢ CUỐI CÙNG: {final_result}")
        print(f"{'='*60}\n")
    
    return final_result


# ========================
# MINH HỌA TÍNH GIÁ TRỊ
# ========================

print("╔" + "═"*58 + "╗")
print("║" + " MINH HỌA TÍNH GIÁ TRỊ BIỂU THỨC HẬU TỐ ".center(58) + "║")
print("╚" + "═"*58 + "╝")

# Ví dụ 1: 3 + 4 * 2 = 3 + 8 = 11
postfix1 = infix_to_postfix("3 + 4 * 2")
evaluate_postfix(postfix1, verbose=True)

# Ví dụ 2: (1 + 2) * 3 = 3 * 3 = 9
postfix2 = infix_to_postfix("( 1 + 2 ) * 3")
evaluate_postfix(postfix2, verbose=True)

# Ví dụ 3: 5! + 3 = 120 + 3 = 123
postfix3 = infix_to_postfix("5! + 3")
evaluate_postfix(postfix3, verbose=True)

# Ví dụ 4: sqrt(16) + 2^3 = 4 + 8 = 12
postfix4 = infix_to_postfix("sqrt(16) + 2 ^ 3")
evaluate_postfix(postfix4, verbose=True)

# Ví dụ 5: sin(30) + cos(60) ≈ 0.5 + 0.5 = 1.0
postfix5 = infix_to_postfix("sin(30) + cos(60)")
evaluate_postfix(postfix5, verbose=True)

---

### 5.1. Tổng kết

Thuật toán **Shunting-yard** cho phép chuyển đổi biểu thức từ dạng trung tố sang hậu tố một cách hiệu quả:

- **Độ phức tạp thời gian:** O(n) - duyệt qua mỗi token đúng một lần
- **Độ phức tạp không gian:** O(n) - cần Stack và Output Queue

### 5.2. Ứng dụng thực tế

- **Máy tính bỏ túi HP:** Sử dụng ký pháp Ba Lan ngược (RPN)
- **Trình biên dịch:** Chuyển đổi biểu thức để sinh mã máy
- **Bảng tính (Excel):** Tính toán công thức trong các ô